In [1]:
import mne
from ni_eeg_forward_project.util import get_labels_of_points, downsample_leadfield_matrix
import numpy as np

In [2]:
fwd = mne.read_forward_solution("../../neurolib/data/datasets/eeg_fsaverage/fsaverage_fwd_sol/fsaverage_surface_src_fixed_orientation-fwd.fif")

fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)

Reading forward solution from /Users/martin/Documents/Uni_MSc/Semester04/ni_project/neurolib/ni_eeg_forward_project/playground_dev/../../neurolib/data/datasets/eeg_fsaverage/fsaverage_fwd_sol/fsaverage_surface_src_fixed_orientation-fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read EEG forward solution (8196 sources, 94 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]


In [3]:
if len(fwd_fixed["src"]) == 2:

    # 0 is left, 1 is right hemisphere
    lh = fwd_fixed['src'][0]
    dip_pos_lh = np.vstack(lh['rr'][lh['vertno']])
    rh = fwd_fixed['src'][1]
    dip_pos_rh = np.vstack(rh['rr'][rh['vertno']])

    if dip_pos_lh.shape != dip_pos_rh.shape:
        raise ValueError("Hemispheres ...")

    dip_pos = np.vstack((dip_pos_lh, dip_pos_rh))
else:
    raise ValueError("Other format than expected for a surface source space for whole brain.")

In [4]:
trans_fsaverage = mne.read_trans("/Users/martin/Documents/Uni_MSc/Semester04/ni_project/neurolib/neurolib/data/datasets/"
                           "eeg_fsaverage/fsaverage/bem/fsaverage-trans.fif")
dip_pos_mni = mne.head_to_mni(dip_pos, subject="fsaverage", mri_head_t=trans_fsaverage)

In [5]:
leadfield = fwd_fixed['sol']['data']

assert leadfield.shape[1] == dip_pos_mni.shape[0]

In [6]:
points_found, label_codes, label_strings = get_labels_of_points(dip_pos_mni, atlas="aal2")

In [16]:
unique_labels, downsampled_leadfield = downsample_leadfield_matrix(leadfield, label_codes)
print(unique_labels.size)
print(downsampled_leadfield.shape)

93
(94, 93)


In [17]:
import matplotlib.pyplot as plt

In [18]:
plt.imshow(downsampled_leadfield,cmap="gray")

NameError: name 'downsample_leadfield' is not defined

In [19]:
np.save("default_leadfield", downsampled_leadfield)

In [21]:
len(fwd_fixed["sol"]["row_names"])

94

In [25]:
dip_pos_mni_lh = mne.head_to_mni(dip_pos_lh, subject="fsaverage", mri_head_t=trans_fsaverage)
points_found, label_codes_lh, label_strings = get_labels_of_points(dip_pos_mni_lh, atlas="aal2")
np.unique(label_codes_lh).size

48

In [26]:
dip_pos_mni_rh = mne.head_to_mni(dip_pos_rh, subject="fsaverage", mri_head_t=trans_fsaverage)
points_found, label_codes_rh, label_strings = get_labels_of_points(dip_pos_mni_rh, atlas="aal2")
np.unique(label_codes_rh).size

50

In [27]:
np.intersect1d(label_codes_lh, label_codes_rh)

array([   0., 4002., 4022., 5001.])

In [ ]:
# it seems to be the case that three medial regions